# Import libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import email
import seaborn as sns
import tensorflow as tf
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import re
import pickle
import joblib
import email

In [ ]:
#load the raw data
df = pickle.load(open('sample_data.pkl' , 'rb'))
df.head()

,file,message
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...
5,allen-p/_sent_mail/1002.,Message-ID: <30965995.1075863688265.JavaMail.e...
6,allen-p/_sent_mail/1003.,Message-ID: <16254169.1075863688286.JavaMail.e...
8,allen-p/_sent_mail/101.,Message-ID: <20641191.1075855687472.JavaMail.e...


# **Utility Function**

In [ ]:
def mail(msg):
  e = email.message_from_string(msg)
  return e.get_payload()

In [ ]:
df['message'] = df['message'].apply(mail)
df.head()

,file,message
1,allen-p/_sent_mail/10.,Traveling to have a business meeting takes the...
3,allen-p/_sent_mail/1000.,"Randy,\n\n Can you send me a schedule of the s..."
5,allen-p/_sent_mail/1002.,"Greg,\n\n How about either next Tuesday or Thu..."
6,allen-p/_sent_mail/1003.,Please cc the following distribution list with...
8,allen-p/_sent_mail/101.,1. login: pallen pw: ke9davis\n\n I don't thi...


In [ ]:
#here we apply all the cleaning things
def data_cleaning(text):
    if text is not None:
        #stopwords_list = stopwords.words('english')
        q = re.sub('Message-ID[^\n]+', ' ', text)
        q = re.sub('Date[^\n]+', ' ', q)
        q = re.sub('X[^\n]+', ' ', q)
        q = re.sub('Content[^\n]+', ' ', q)
        q = re.sub('From[^\n]+', ' ', q)
        q = re.sub('To[^\n]+', ' ', q)
        q = re.sub('Subject[^\n]+', ' ', q)
        q = re.sub('Mime-Version[^\n]+', ' ', q)
        q = re.sub('.*?\(.*?\)',' ',q)
        q = re.sub('--[^\n]+', ' ', q)
        q = re.sub('cc:[^\n]+|Cc:[^\n]+', ' ', q)
        q = re.sub('Sent:[^\n]+', '', q)
        q = re.sub('email address:[^\n]+', ' ', q)
        q = re.sub('[0-9]+', ' ', q)
        q = re.sub('[\/:]', ' ', q)
        q = re.sub('AM|PM|a\.m\.|p\.m\.', ' ', q)
        q = re.sub('\(|\)|,|;|\.|!', ' ', q)
        q = re.sub('-[^\n]+', '', q)
        q = re.sub('-', ' ', q)
        q = re.sub('\?', ' ', q)
        q = re.sub('\@', ' ', q)
        q = re.sub('[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+', ' ', q)
        q = re.sub('(Write to:|From:).*',' ',q)
        q = re.sub('re:', ' ', q)
        q = re.sub('\[[^]]*\]', ' ', q)
        q = re.sub(r'[^\w\s]',' ',q)
        #remove the < anykeyword> like this from the data
        q = re.sub('<.*>',' ',q)
        #remove the all word which is present in the bracket
        q = re.sub('\(.*\)',' ',q)
        #remove all the new lines
        q = re.sub(r'[\n\t-]*','',q)

        #remove all the word which is end with :
        q = re.sub(r'\w+:\s?','',q)

        #replace the short word with there full words code taken from donors choose assignment
            # specific
        q = re.sub(r"won't", " will not", q)
        q = re.sub(r"can\'t", " can not", q)

        # general
        q = re.sub(r"n\'t", " not", q)
        q = re.sub(r"\'re", " are", q)
        q = re.sub(r"\'s", " is", q)
        q = re.sub(r"\'d", " would", q)
        q = re.sub(r"\'ll", " will", q)
        q = re.sub(r"\'t", " not", q)
        q = re.sub(r"\'ve", " have", q)
        q = re.sub(r"\'m", " am", q)

        q = re.sub(r'\b_([a-zA-z]+)_\b',r'\1' , q)
        q = re.sub(r'\b_([a-zA-z]+)\b',r'\1' , q)
        q = re.sub(r'\b([a-zA-z]+)_\b',r'\1' , q)

        #remove _ sign from the word like di_shivam and we want only shivam
        q = re.sub(r'\b[a-zA-Z]{1}_([a-zA-Z]+)',r'\1'  , q)
        q = re.sub(r'\b[a-zA-Z]{2}_([a-zA-Z]+)',r'\1' , q)

        #convert all into a lower case
        q = q.lower()

        #remove the words which are greater than or equal to 15 or less than or equal to 2
        #https://stackoverflow.com/questions/24332025/remove-words-of-length-less-than-4-from-string
        #q = re.sub(r'\b\w{,2}\b' ,'', q)
        q = re.sub(r'\b\w{15,}\b','', q)

        #replace all the words except "A-Za-z_" with space
        q= re.sub(r'[^a-zA-Z_]',' ',q)
        q = q.replace("\n", " ")
#       f = ' '.join([word for word in q.split() if word not in stopwords.words("english")])\
        f = ' '.join([word for word in q.split()])
        return f

In [ ]:
df['message'] = df['message'].apply(data_cleaning)
df.head()

,file,message
1,allen-p/_sent_mail/10.,traveling to have a business meeting takes the...
3,allen-p/_sent_mail/1000.,randy can you send me a schedule of the salary...
5,allen-p/_sent_mail/1002.,greg how about either next tuesday or thursday...
6,allen-p/_sent_mail/1003.,please cc the following distribution list with...
8,allen-p/_sent_mail/101.,login pallen pw ke davis i don t think these a...


In [ ]:
def data(c):
  x = []
  y = []
  for i in tqdm((c)):
    a = i.split()
    for j in range(1,len(a)):
          x.append(' '.join(a[:j]))
          y.append(' '.join(a[j:]))
  train_set = pd.DataFrame(data={'x_train':x,'y_train':y})
  train_set['x_train_len'] = train_set['x_train'].apply(lambda text:len(text.split()))
  train_set['y_train_len'] = train_set['y_train'].apply(lambda text:len(text.split()))
  train_set = train_set[train_set['x_train_len']>3]
  train_set = train_set[train_set['x_train_len']<30]
  train_set = train_set[train_set['y_train_len']<8]
  data = train_set

  data['enc_inp'] = '<start> ' + data['x_train'].astype(str)  + ' <end>'
  data['dec_inp'] = '<start> ' + data['y_train'].astype(str)
  data['our'] = data['y_train'].astype(str) + ' <end>'
  data = data.drop(['x_train','y_train','x_train_len', 'y_train_len'], axis=1)
  return data

In [ ]:
d = data(df['message'])

100%|██████████| 1500/1500 [00:01<00:00, 1382.35it/s]


In [ ]:
d.head(4)

,enc_inp,dec_inp,our
133,<start> randy can you send me a schedule of th...,<start> changes that need to be made phillip,changes that need to be made phillip <end>
134,<start> randy can you send me a schedule of th...,<start> that need to be made phillip,that need to be made phillip <end>
135,<start> randy can you send me a schedule of th...,<start> need to be made phillip,need to be made phillip <end>
136,<start> randy can you send me a schedule of th...,<start> to be made phillip,to be made phillip <end>


In [ ]:
d.enc_inp.values[0]

'<start> randy can you send me a schedule of the salary and level of everyone in the scheduling group plus your thoughts on any <end>'

# **Loding Model**

In [ ]:
ita_token = joblib.load('/content/drive/MyDrive/model_new/ita_token.pkl')
eng_token = joblib.load('/content/drive/MyDrive/model_new/eng_token.pkl')
y_token   = joblib.load('/content/drive/MyDrive/model_new/y_token.pkl')

In [ ]:
input_vocab  = len(ita_token.word_index) + 1
output_vocab = len(y_token.word_index) + 1
print('input vocab size ==>' , input_vocab)
print('output vocab size ==>' , output_vocab)

input vocab size ==> 18547
output vocab size ==> 12127


In [ ]:
class Encoder(tf.keras.Model):
    '''
    Encoder model -- That takes a input sequence and returns output sequence
    '''

    def __init__(self,inp_vocab_size,embedding_size,lstm_size,input_length):

        #Initialize Embedding layer
        #Intialize Encoder LSTM layer
        super().__init__()
        self.inp_vocab_size = inp_vocab_size
        self.embedding_size = embedding_size
        self.input_length = input_length
        self.lstm_size= lstm_size
        self.embedding = tf.keras.layers.Embedding(input_dim=self.inp_vocab_size, output_dim=self.embedding_size, name="embedding_layer_encoder")
        self.lstm = tf.keras.layers.LSTM(self.lstm_size, return_state=True, return_sequences=True, name="Encoder_LSTM", recurrent_initializer='glorot_uniform')

    def call(self,input_sequence,initial_state):
      '''
          This function takes a sequence input and the initial states of the encoder.
          Pass the input_sequence input to the Embedding layer, Pass the embedding layer ouput to encoder_lstm
          returns -- All encoder_outputs, last time steps hidden and cell state
      '''
      input_embedd = self.embedding(input_sequence)
      lstm_output, lstm_state_h, lstm_state_c = self.lstm(input_embedd,initial_state = initial_state)
      return lstm_output, lstm_state_h, lstm_state_c
    
    def initialize_states(self,batch_size):
      '''
      Given a batch size it will return intial hidden state and intial cell state.
      If batch size is 32- Hidden state shape is [32,lstm_units], cell state shape is [32,lstm_units]
      '''
      return tf.zeros((batch_size, self.lstm_size)) , tf.zeros((batch_size, self.lstm_size))

In [ ]:
class Decoder(tf.keras.Model):
    '''
    Encoder model -- That takes a input sequence and returns output sequence
    '''

    def __init__(self,out_vocab_size,embedding_size,lstm_size,input_length):
      super().__init__()
      self.out_vocab_size = out_vocab_size
      self.embedding_size = embedding_size
      self.input_length = input_length
      self.lstm_size= lstm_size

        #Initialize Embedding layer
        #Intialize Decoder LSTM layer

      self.embedding = tf.keras.layers.Embedding(input_dim = self.out_vocab_size, output_dim = self.embedding_size, name="embedding_decoder_layer")
      
      self.lstm = tf.keras.layers.LSTM(self.lstm_size, return_state=True, return_sequences=True, name="Decoder_lstm_layer", recurrent_initializer='glorot_uniform')
        


    def call(self,input_sequence,initial_states):


      
      input_embedd = self.embedding(input_sequence)
      decoder_output, decoder_final_state_h, decoder_final_state_c = self.lstm(input_embedd,initial_state = initial_states)
      return decoder_output, decoder_final_state_h, decoder_final_state_c

In [ ]:
class Encoder_decoder(tf.keras.Model):
    
    def __init__(self,encoder_inputs_length,decoder_inputs_length, output_vocab_size):
        super().__init__() # https://stackoverflow.com/a/27134600/4084039
        #(self,inp_vocab_size,embedding_size,lstm_size,input_length):
        self.encoder = Encoder(inp_vocab_size = input_vocab, embedding_size =  256, lstm_size = 200, input_length= encoder_inputs_length)
        self.decoder = Decoder(out_vocab_size = output_vocab, embedding_size = 256, lstm_size = 200, input_length= decoder_inputs_length)
        self.dense   = Dense(output_vocab_size, activation='softmax')
        
        #Create encoder object
        #Create decoder object
        #Intialize Dense layer(out_vocab_size) with activation='softmax'
    
    
    def call(self,data):
        '''
        A. Pass the input sequence to Encoder layer -- Return encoder_output,encoder_final_state_h,encoder_final_state_c
        B. Pass the target sequence to Decoder layer with intial states as encoder_final_state_h,encoder_final_state_C
        C. Pass the decoder_outputs into Dense layer 
        
        Return decoder_outputs
        '''
        input_to_encoder,input_to_decoder = data[0], data[1]
        initial_state= self.encoder.initialize_states(tf.shape(input_to_encoder)[0])
        encoder_output,state_h,state_c = self.encoder(input_to_encoder,initial_state)

        decoder_output, decoder_final_state_h, decoder_final_state_c = self.decoder(input_to_decoder, [state_h,state_c]) 
        decoder_output = self.dense(decoder_output)

        # return the decoder output
        return decoder_output

In [ ]:
from tensorflow.keras.layers import Embedding, LSTM, Dense, Activation , add
model  = Encoder_decoder(encoder_inputs_length=31 ,decoder_inputs_length=8 , output_vocab_size = output_vocab)

In [ ]:
model.build(input_shape=[(None,31),(None,8)])
model.summary()

Model: "encoder_decoder_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_1 (Encoder)         multiple                  5113632   
                                                                 
 decoder_1 (Decoder)         multiple                  3470112   
                                                                 
 dense_1 (Dense)             multiple                  2437527   
                                                                 
Total params: 11,021,271
Trainable params: 11,021,271
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#Load weights
model.load_weights('/content/drive/MyDrive/model_new/best_model1.h5')

## Predictions

In [ ]:
#here we change the key value pair to value key pair
eng_word = y_token.word_index
new_dict = dict([(value, key) for key, value in eng_word.items()])
new_dict[0] = 'start'

In [ ]:
def predict(input_sentence):
# A. Given input sentence, convert the sentence into integers using tokenizer used earlier
#here we also check the we have the start and end token in sentences
#then we convert out input into to token and do a padding
  input = input_sentence
  if input.split(' ')[0] != '<start>' and input.split(' ')[-1] != '<end>':
    input = '<start>'+ ' ' + input + ' ' + '<end>'
  else:
    input = str(input)
  input = ita_token.texts_to_sequences([str(input)])
  input = pad_sequences(input, padding="post",maxlen= 31)

#as we know we have a three layer 1 ==> encoder 2 ==> decoder 3 ==> dense
#so first is encoder we give the input as input sec and ini_state

  enc_ini_states = model.layers[0].initialize_states(1)
  enc_out, enc_h_state, enc_c_state = model.layers[0](input, enc_ini_states)

#output of the encoder is a input as decoder
# first word input of decoder is start token 
# then 2nd input of decoder is predecting word by model

  decoder_initial_state = [enc_h_state, enc_c_state]
  decoder_initial_input = np.zeros((1,1))
  decoder_initial_input[0,0] = eng_token.word_index['<start>']

  predict_word = []
  w = []
  for i in range(8):
    dec_out, dec_h_state, dec_c_state = model.layers[1](decoder_initial_input, decoder_initial_state)
    # we use 3rd layer and we get the max proba word as out put and the this word is next input of the decoder 
    english_predict = np.argmax(model.layers[2](dec_out).numpy().ravel())
    predict_word.append(english_predict)
    decoder_initial_input[0,0] = english_predict
    #replacing the next decoder initial states with current decoder output 
    decoder_initial_state = [dec_h_state, dec_c_state]
    w.append(new_dict[english_predict])

    if new_dict[english_predict] == '<end>':
      break
  return ' '.join(w)

In [ ]:
predict('if you have ')

'any questions please call me <end>'

In [ ]:
print(d.enc_inp.values[0])
print(predict(d.enc_inp.values[0]))

<start> randy can you send me a schedule of the salary and level of everyone in the scheduling group plus your thoughts on any <end>
changes <end>


# **Function 1**

In [ ]:
def function(df):
  predicted_out = []
  predicted_input = []
  
  if type(df) != str: 
    df = df.apply(mail)
    df = df.apply(data_cleaning)
    df = data(df)
    
    for i in df.enc_inp.values:
      predicted_input.append(i)
      predicted_out.append(predict(i))
  else:
    predicted_input.append(df)
    predicted_out.append(predict(df))

  new_df = pd.DataFrame()
  new_df['Input'] = predicted_input
  new_df['Predicted'] = predicted_out

  return new_df

In [ ]:
output = function(df['message'])
output.head(10)

100%|██████████| 1500/1500 [00:01<00:00, 1380.84it/s]


,Input,Predicted
0,<start> randy can you send me a schedule of th...,changes <end>
1,<start> randy can you send me a schedule of th...,to the agency agreement <end>
2,<start> randy can you send me a schedule of th...,issue should be built soon <end>
3,<start> randy can you send me a schedule of th...,to be made hold of them <end>
4,<start> randy can you send me a schedule of th...,be made available on a <end>
5,<start> randy can you send me a schedule of th...,killed <end>
6,<start> randy can you send me a schedule of th...,on <end>
7,<start> greg how about either <end>,of the west <end>
8,<start> greg how about either next <end>,last week or the prices said that they
9,<start> greg how about either next tuesday <end>,past <end>


In [ ]:
function('if you have  ')

,Input,Predicted
0,if you have,any questions please call me <end>


In [ ]:
function('please call me')

,Input,Predicted
0,please call me,if you have any questions or comments <end>


# **Function 2**

In [ ]:
def function_2(df):
  df = df.apply(mail)
  df = df.apply(data_cleaning)
  df = data(df)

  bleu_score = []
  for i in (range(2000)):
    actual  = df.our.iloc[i] 
    Predict = predict(df.enc_inp.iloc[i])
    bleu_score.append(bleu.sentence_bleu([actual.split(),], Predict.split()))

  return('Bleu Score ==> ', np.mean(bleu_score))

In [ ]:
import nltk.translate.bleu_score as bleu
function_2(df['message'])

100%|██████████| 79616/79616 [00:07<00:00, 10993.59it/s]
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


('Bleu Score ==> ', 0.7099411677193744)